# Autor
Notebook criado por [Wallacy Pasqualini](https://www.linkedin.com/in/wallacypasqualini/).

Visite meu [GitHub](https://github.com/WallPasq) para ver meus projetos.

# O problema

> 💡 O objetivo desse Notebook está em mostrar o poder do Z-score. Leia até o final para entender o que foi feito em cada célula.

> ❗️ Abaixo conto uma história inspirada em algo que aconteceu comigo, com detalhes e exemplos fictícios para anonimizar devidamente os dados.

Recentemente passei por um desafio onde me foi passado uma meta de Ticket Médio geral, e eu precisava verificar o quanto o Ticket Médio de cada canal variou em relação à meta. 

Parece uma tarefa bem simples a princípio, certo? Só que tive dois problemas logo no início:

1. Os canais estavam segmentados em três colunas, onde cada coluna definia uma especificação diferente. Exemplo:

    | Tipo   | Canal      | Direcionamento   |
    |--------|------------|------------------|
    | Ads    | Facebook   | Blog             | 
    
    <br />
1. O Ticket Médio mudava muito dependendo do canal, o que deixava alguns muito acima da meta e outros muito abaixo. Exemplo:

    | Tipo   | Canal      | Direcionamento   | Meta     | Ticket Médio | Desvio |
    |--------|------------|------------------|----------|--------------|--------|
    | Ads    | Facebook   | Blog             | R$ 6.000 | R$ 10.000    | 67%    |
    | E-mail | Campanha   | Site             | R$ 6.000 | R$ 400       | - 93%  |

    <br />

O que você faria, nesse caso? Entregaria os números dessa forma?

Eu aprendi com o professor Daniel, na [Data Science Academy](https://www.datascienceacademy.com.br/), que o valor de um profissional na área de Dados está na sua capacidade de reconhecer e resolver problemas.

Por isso, para mim se tornou inaceitável entregar os valores dessa forma, pois estavam longe de representar a realidade.

Mas, como ajustar a meta de Ticket Médio para cada canal?

Talvez calcular um fator, que seria o Ticket Médio de cada combinação das três colunas dividido pelo Ticket Médio geral.

Depois bastaria multiplicar esse fator pela Meta, obtendo assim a Meta ajustada.

Essa seria uma possível solução, se não fosse o fato de eu precisar levar esses dados para o Power BI, permitindo filtros de seleção múltipla por cada coluna, que poderiam ser ou não combinados.

E agora? Talvez calcular o desvio no nível de cada coluna e também nos níveis das combinações?

Mas e se o número de colunas aumentasse depois? Certamente essa solução não seria viável nem escalável.

Isso além do fato da tabela ter milhões de linhas, o que prejudicaria a performance no Power BI para calcular o Ticket Médio de cada combinação e dividir pelo Ticket Médio geral, mesmo utilizando o contexto de filtro.

Não consegui encontrar uma solução naquele momento. Contudo, meu amigo [Gabriel Fonseca](https://www.linkedin.com/in/gabriel-felipe-fonseca/) me ensinou o seguinte:

> Às vezes é melhor dormir com a informação e deixar a nossa mente "pensar durante o repouso".

E foi o que eu fiz!

# Propriedades do Z-score

Você já ouviu falar no Z-score? Em uma definição simples, ele é a quantidade de desvios padrão que um valor está acima ou abaixo da média.

Por exemplo, em uma distribuição onde a média é 6.000 e o desvio padrão é 1.000, o valor 8.000 possui Z-score 2. Isso pois ele está 2 desvios padrão acima da média.

Já o valor 5.000 possui Z-score -1, pois ele está 1 desvio padrão abaixo da média.

Pegou a ideia? Sua fórmula é bem simples:

$$
\text{Z-score} = \frac{\text{Valor} - \text{Média}}{\text{Desvio padrão}}
$$

Dada sua fórmula, ele possui uma característica muito interessante: a soma de todos os Z-scores de um conjunto de dados é igual a 0.

> 💡 Isso se dá pela 1ª Propriedade da Média Aritmética, que diz que a soma dos desvios de cada valor em relação à média aritmética de um conjunto de dados é sempre igual a 0.

> 🧠 Não sabia disso? Bom, agora você sabe um dos motivos pelos quais elevamos os desvios ao quadrado para calcular o desvio padrão.

Se você já conhecia o Z-score e essa propriedade, um ponto que talvez você não sabia é que a média de um subconjunto de valores pode ser encontrada da seguinte forma:

$$
\text{Média do subconjunto} = \text{Média do conjunto} \cdot \text{Desvio padrão do conjunto} \cdot \text{Z-score médio do subconjunto}
$$

Onde o Z-score médio do subconjunto é igual a:

$$
\text{Z-score médio do subconjunto} = \frac{1}{\text{n}} \cdot \sum_{i=1}^{n}{\text{Z-score}_{i}}
\\~\\
\text{n} = \text{Quantidade de elementos do subconjunto}
$$

Essa propriedade é muito importante para a solução que apresentarei, e é demonstrada abaixo neste Notebook, no capítulo: [Encontrando a média de cada subconjunto com Z-score](#capitulo1).

> 💡 Note que estamos trazendo uma solução bem mais complexa e, ao mesmo tempo, mais plausível, do que simplesmente calcular o fator utilizando o Ticket Médio de cada canal.

# A solução

Se o Z-score médio pode ser utilizado para calcular a média de um subconjunto, ele também pode ser utilizado para ajustar a meta para cada subconjunto.

Contudo, calcular o Z-score médio para cada subconjunto cai na mesma problemática que levantei acima, principalmente de performance no Power BI.

Além, onde eu poderia calcular esse Z-score? Nos valores atuais? Isso não seria justo, pois a meta foi definida antes desses valores sequer serem gerados.

Assim, a solução encontrada foi levar alguns dados prontos para o Power BI, e realizar os demais cálculos lá, da seguinte forma:

1. Dados prontos:

    1. Calcular a soma dos Z-scores e a quantidade de elementos de cada combinação das três colunas através de dados históricos, por exemplo, do último mês:

        | Tipo   | Canal      | Direcionamento   | Soma Z-scores | Quantidade |
        |--------|------------|------------------|---------------|------------|
        | Ads    | Facebook   | Blog             | 100.000       | 75.000     |
        | E-mail | Campanha   | Site             | - 300.000     | 125.000    |

        <br />
    1. Calcular o Coeficiente de Variação dos dados históricos:

        $$
        \text{Coeficiente de Variação} = \frac{\text{Desvio padrão}}{\text{Média}}
        $$
    
    1. Calcular a variação da meta:

        $$
        \text{Variação da Meta} = \text{Meta} \cdot \text{Coeficiente de Variação}
        $$

1. Power BI:

    1. Ajustar a meta do Ticket Médio para cada canal:

        $$
        \text{Meta ajustada} = \text{Meta} + \text{Variação da Meta} \cdot \frac{\sum_{i=1}^{n}{\text{Soma Z-score}_{i}}}{\sum_{i=1}^{n}{\text{Quantidade}_{i}}}
        $$

        <br />

        Em DAX:

        ```language-dax
        meta_ajustada = DISTINCT(fMeta[meta]) + 
                        DISTINCT(fMeta[variacao]) *
                        DIVIDE(
                            SUM(fBaseHistorica[soma_z_score],
                            SUM(fBaseHistorica[quantidade]),
                            0))
        ```
        
        > 🧠 Note que a soma dos Z-scores dividido pela soma das quantidades nada mais é do que o Z-score médio do subconjunto.

Como ao invés da tabela inteira levamos uma tabela sumarizada para o Power BI, reduzimos drasticamente o número de linhas dela.

Abaixo, no exemplo deixado neste Notebook, você verá que uma sumarização parecida reduz uma tabela de 5 milhões de linhas para menos de 18 mil.

> 🎉 Uma redução de 99,6%!

Além disso, os cálculos deixados para o Power BI são executados rapidamente utilizando o contexto de filtro.

# Prática

Agora eu te convido a executar esse Notebook e acompanhar, através dos exemplos com dados aleatórios, o que foi feito e como a meta foi ajustada para cada canal.

Além, tente mudar os parâmetros da função base_aleatoria e analisar os resultados gerados.

Espero que você tenha aprendido algo comigo, e que possa levar esse aprendizado para seu futuro!

<a id='capitulo1'></a>
# Encontrando a média de cada subconjunto com Z-score

In [1]:
# Importa as bibliotecas
import pandas as pd
import numpy as np
import string
import time
from typing import List

In [2]:
# Define uma função para criar uma base aleatória com valores outliers
def base_aleatoria(tamanho: int = 5_000_000,
                   percentual_outliers_baixo: float = 0.05,
                   percentual_outliers_alto: float = 0.05,
                   classes_outliers_baixo: List[str] = [['Y1', 'Y2', 'Y3'],
                                                        ['Z1', 'Z2', 'Z3']],
                   classes_outliers_alto: List[str] = [['A1', 'A2', 'A3'],
                                                       ['B1', 'B2', 'B3'],
                                                       ['C1', 'C2', 'C3'],
                                                       ['D1', 'D2', 'D3']],
                   media_normal: float = 6_000,
                   desvio_padrao_normal: float = 500,
                   media_outlier_baixo: int = 2_000,
                   desvio_padrao_outlier_baixo: int = 100,
                   media_outlier_alto: int = 10_000,
                   desvio_padrao_outlier_alto: int = 2_000) -> pd.DataFrame:
    
    horario_inicio = time.time()
    letras = list(string.ascii_uppercase)
    
    quantidade_outliers_baixo = int(tamanho * percentual_outliers_baixo)
    quantidade_outliers_alto = int(tamanho * percentual_outliers_alto)
    tamanho_sem_outliers = tamanho - quantidade_outliers_baixo - quantidade_outliers_alto
    quantidade_outliers_baixo_por_classe = quantidade_outliers_baixo // len(classes_outliers_baixo)
    quantidade_outliers_alto_por_classe = quantidade_outliers_alto // len(classes_outliers_alto)

    print(f'Criando os primeiros {tamanho_sem_outliers:_.0f} valores'.replace('.', ',').replace('_', '.'))
    classe_1 = np.char.add(np.random.choice(letras, tamanho_sem_outliers), '1')
    classe_2 = np.char.add(np.random.choice(letras, tamanho_sem_outliers), '2')
    classe_3 = np.char.add(np.random.choice(letras, tamanho_sem_outliers), '3')
    valores = np.around(np.random.normal(media_normal, desvio_padrao_normal, tamanho_sem_outliers), 2)
    horario_fim_1 = time.time()
    print(f'Valores criados em {(horario_fim_1 - horario_inicio):_.2f} segundos'.replace('.', ',').replace('_', '.'))

    for i, cl in enumerate(classes_outliers_baixo, 1):
        print(f'\nCriando os {quantidade_outliers_baixo_por_classe:_.0f} outliers com valores baixos do {i}º conjunto de classes'.replace('.', ',').replace('_', '.'))
        classe_1 = np.concatenate((classe_1, np.repeat(cl[0], quantidade_outliers_baixo_por_classe)), axis=None)
        classe_2 = np.concatenate((classe_2, np.repeat(cl[1], quantidade_outliers_baixo_por_classe)), axis=None)
        classe_3 = np.concatenate((classe_3, np.repeat(cl[2], quantidade_outliers_baixo_por_classe)), axis=None)
        valores = np.concatenate((valores, np.around(np.random.normal(media_outlier_baixo, desvio_padrao_outlier_baixo, quantidade_outliers_baixo_por_classe), 2)), axis=None)
        horario_fim_2 = time.time()
        print(f'Valores criados em {(horario_fim_2 - horario_fim_1):_.2f} segundos'.replace('.', ',').replace('_', '.'))
        horario_fim_1 = horario_fim_2

    for i, cl in enumerate(classes_outliers_alto, 1):
        print(f'\nCriando os {quantidade_outliers_alto_por_classe:_.0f} outliers com valores altos do {i}º conjunto de classes'.replace('.', ',').replace('_', '.'))
        classe_1 = np.concatenate((classe_1, np.repeat(cl[0], quantidade_outliers_alto_por_classe)), axis=None)
        classe_2 = np.concatenate((classe_2, np.repeat(cl[1], quantidade_outliers_alto_por_classe)), axis=None)
        classe_3 = np.concatenate((classe_3, np.repeat(cl[2], quantidade_outliers_alto_por_classe)), axis=None)
        valores = np.concatenate((valores, np.around(np.random.normal(media_outlier_alto, desvio_padrao_outlier_alto, quantidade_outliers_alto_por_classe), 2)), axis=None)
        horario_fim_2 = time.time()
        print(f'Valores criados em {(horario_fim_2 - horario_fim_1):_.2f} segundos'.replace('.', ',').replace('_', '.'))
        horario_fim_1 = horario_fim_2

    base = pd.DataFrame(dict(classe_1=classe_1, classe_2=classe_2, classe_3=classe_3, valor=valores))
    print(f'\nBase criada em {(horario_fim_2 - horario_inicio):.2f} segundos'.replace('.', ',').replace('_', '.'))
    return base

In [3]:
# Gera a base histórica aleatória
base_historica = base_aleatoria()

# Obtém algumas estatísticas da base
quantidade_linhas_base_historica = len(base_historica)
media_base_historica = base_historica.valor.mean()
desvio_padrao_base_historica = base_historica.valor.std()
coeficiente_variacao_base_historica = desvio_padrao_base_historica / media_base_historica

# Calcula o Z-score de cada valor
base_historica['z_score'] = (base_historica.valor - media_base_historica) / desvio_padrao_base_historica
display(base_historica)

Criando os primeiros 4.500.000 valores
Valores criados em 8,58 segundos

Criando os 125.000 outliers com valores baixos do 1º conjunto de classes
Valores criados em 0,09 segundos

Criando os 125.000 outliers com valores baixos do 2º conjunto de classes
Valores criados em 0,10 segundos

Criando os 62.500 outliers com valores altos do 1º conjunto de classes
Valores criados em 0,10 segundos

Criando os 62.500 outliers com valores altos do 2º conjunto de classes
Valores criados em 0,11 segundos

Criando os 62.500 outliers com valores altos do 3º conjunto de classes
Valores criados em 0,11 segundos

Criando os 62.500 outliers com valores altos do 4º conjunto de classes
Valores criados em 0,21 segundos

Base criada em 9,31 segundos


,classe_1,classe_2,classe_3,valor,z_score
0,U1,P2,L3,5555.58,-0.312557
1,T1,I2,J3,6409.78,0.287653
2,V1,M2,T3,5690.90,-0.217474
3,R1,G2,R3,6041.22,0.028681
4,M1,I2,S3,6134.64,0.094324
...,...,...,...,...,...
4999995,D1,D2,D3,7805.76,1.268550
4999996,D1,D2,D3,9972.28,2.790872
4999997,D1,D2,D3,11399.51,3.793727
4999998,D1,D2,D3,11718.16,4.017629


In [4]:
# Mostra a quantidade de Linhas, a média geral do Valor e a soma do Z-score (a soma tem que ser zero)
print(f'Quantidade de Linhas: {quantidade_linhas_base_historica:_.0f}'.replace('.', ',').replace('_', '.'))
print(f'Média do Valor: {media_base_historica:_.2f}'.replace('.', ',').replace('_', '.'))
print(f'Soma do Z-score: {base_historica.z_score.sum():_.2f}'.replace('.', ',').replace('_', '.'))

Quantidade de Linhas: 5.000.000
Média do Valor: 6.000,40
Soma do Z-score: -0,00


In [5]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_1
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_classe_1 = base_historica.groupby('classe_1').agg({'z_score': 'sum', 'classe_1': 'count', 'valor': 'mean'})
analise_classe_1.rename(columns={'z_score': 'soma_z_score', 'classe_1': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_1.reset_index(inplace=True)
analise_classe_1['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1.soma_z_score / analise_classe_1.quantidade)
analise_classe_1.insert(3, 'proporcao', analise_classe_1.quantidade / quantidade_linhas_base_historica)
analise_classe_1.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_classe_1.proporcao * analise_classe_1.media_real).sum()
resultado = 'ela é igual à média da base histórica' if media_ponderada.round(2) == media_base_historica.round(2) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_classe_1.soma_z_score.sum()
soma_quantidade = analise_classe_1.quantidade.sum()
soma_proporcao = analise_classe_1.proporcao.sum()
analise_classe_1 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_1],
    ignore_index=True)

analise_classe_1['medias_iguais'] = analise_classe_1.media_real.round(0) == analise_classe_1.media_calculada_com_z_score.round(0)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_classe_1.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_classe_1.proporcao = analise_classe_1.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_classe_1.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 6.000,40 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_1,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,"-0,00",5.000.000,"100,00%","6.000,40","6.000,40",True
1,A1,"176.108,41",234.985,"4,70%","7.066,99","7.066,99",True
2,C1,"176.198,38",235.537,"4,71%","7.065,03","7.065,03",True
3,D1,"175.746,75",235.097,"4,70%","7.064,29","7.064,29",True
4,B1,"175.361,98",236.113,"4,72%","7.057,39","7.057,39",True
5,G1,"269,58",172.894,"3,46%","6.002,62","6.002,62",True
6,M1,"111,70",173.646,"3,47%","6.001,32","6.001,32",True
7,E1,"87,19",172.123,"3,44%","6.001,12","6.001,12",True
8,N1,"82,59",172.945,"3,46%","6.001,08","6.001,08",True
9,U1,"79,55",172.067,"3,44%","6.001,06","6.001,06",True


In [6]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_2
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_classe_2 = base_historica.groupby('classe_2').agg({'z_score': 'sum', 'classe_2': 'count', 'valor': 'mean'})
analise_classe_2.rename(columns={'z_score': 'soma_z_score', 'classe_2': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_2.reset_index(inplace=True)
analise_classe_2['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_2.soma_z_score / analise_classe_2.quantidade)
analise_classe_2.insert(3, 'proporcao', analise_classe_2.quantidade / quantidade_linhas_base_historica)
analise_classe_2.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_classe_2.proporcao * analise_classe_2.media_real).sum()
resultado = 'ela é igual à média da base histórica' if media_ponderada.round(2) == media_base_historica.round(2) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_classe_2.soma_z_score.sum()
soma_quantidade = analise_classe_2.quantidade.sum()
soma_proporcao = analise_classe_2.proporcao.sum()
analise_classe_2 = pd.concat([
    pd.DataFrame(dict(classe_2=['Total'],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_2],
    ignore_index=True)

analise_classe_2['medias_iguais'] = analise_classe_2.media_real.round(0) == analise_classe_2.media_calculada_com_z_score.round(0)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_classe_2.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_classe_2.proporcao = analise_classe_2.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_classe_2.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 6.000,40 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_2,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,"-0,00",5.000.000,"100,00%","6.000,40","6.000,40",True
1,C2,"176.235,55",236.190,"4,72%","7.062,31","7.062,31",True
2,D2,"175.777,62",236.081,"4,72%","7.060,04","7.060,04",True
3,A2,"175.763,79",236.079,"4,72%","7.059,97","7.059,97",True
4,B2,"175.503,25",236.019,"4,72%","7.058,67","7.058,67",True
5,O2,"166,49",173.242,"3,46%","6.001,77","6.001,77",True
6,M2,"109,14",173.491,"3,47%","6.001,30","6.001,30",True
7,J2,"69,90",172.945,"3,46%","6.000,98","6.000,98",True
8,W2,"57,38",173.408,"3,47%","6.000,87","6.000,87",True
9,I2,"50,53",172.477,"3,45%","6.000,82","6.000,82",True


In [7]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_3
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_classe_3 = base_historica.groupby('classe_3').agg({'z_score': 'sum', 'classe_3': 'count', 'valor': 'mean'})
analise_classe_3.rename(columns={'z_score': 'soma_z_score', 'classe_3': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_3.reset_index(inplace=True)
analise_classe_3['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_3.soma_z_score / analise_classe_3.quantidade)
analise_classe_3.insert(3, 'proporcao', analise_classe_3.quantidade / quantidade_linhas_base_historica)
analise_classe_3.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_classe_3.proporcao * analise_classe_3.media_real).sum()
resultado = 'ela é igual à média da base histórica' if media_ponderada.round(2) == media_base_historica.round(2) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_classe_3.soma_z_score.sum()
soma_quantidade = analise_classe_3.quantidade.sum()
soma_proporcao = analise_classe_3.proporcao.sum()
analise_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_3=['Total'],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_3],
    ignore_index=True)

analise_classe_3['medias_iguais'] = analise_classe_3.media_real.round(0) == analise_classe_3.media_calculada_com_z_score.round(0)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_classe_3.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_classe_3.proporcao = analise_classe_3.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 6.000,40 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_3,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,"-0,00",5.000.000,"100,00%","6.000,40","6.000,40",True
1,C3,"176.216,87",235.701,"4,71%","7.064,40","7.064,40",True
2,A3,"175.797,85",235.853,"4,72%","7.061,19","7.061,19",True
3,D3,"175.738,64",235.868,"4,72%","7.060,76","7.060,76",True
4,B3,"175.424,44",235.467,"4,71%","7.060,67","7.060,67",True
5,I3,"223,99",172.975,"3,46%","6.002,24","6.002,24",True
6,L3,"125,50",172.513,"3,45%","6.001,44","6.001,44",True
7,X3,"123,32",173.455,"3,47%","6.001,41","6.001,41",True
8,H3,"101,74",173.221,"3,46%","6.001,24","6.001,24",True
9,S3,"69,29",172.599,"3,45%","6.000,97","6.000,97",True


In [8]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_1 + classe_2
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_classe_1_classe_2 = base_historica.groupby(['classe_1', 'classe_2']).agg({'z_score': 'sum', 'classe_1': 'count', 'valor': 'mean'})
analise_classe_1_classe_2.rename(columns={'z_score': 'soma_z_score', 'classe_1': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_1_classe_2.reset_index(inplace=True)
analise_classe_1_classe_2['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1_classe_2.soma_z_score / analise_classe_1_classe_2.quantidade)
analise_classe_1_classe_2.insert(3, 'proporcao', analise_classe_1_classe_2.quantidade / quantidade_linhas_base_historica)
analise_classe_1_classe_2.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_classe_1_classe_2.proporcao * analise_classe_1_classe_2.media_real).sum()
resultado = 'ela é igual à média da base histórica' if media_ponderada.round(2) == media_base_historica.round(2) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_classe_1_classe_2.soma_z_score.sum()
soma_quantidade = analise_classe_1_classe_2.quantidade.sum()
soma_proporcao = analise_classe_1_classe_2.proporcao.sum()
analise_classe_1_classe_2 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_2=[''],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_1_classe_2],
    ignore_index=True)

analise_classe_1_classe_2['medias_iguais'] = analise_classe_1_classe_2.media_real.round(0) == analise_classe_1_classe_2.media_calculada_com_z_score.round(0)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_classe_1_classe_2.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_classe_1_classe_2.proporcao = analise_classe_1_classe_2.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_classe_1_classe_2.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 6.000,40 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_1,classe_2,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,,"-0,00",5.000.000,"100,00%","6.000,40","6.000,40",True
1,C1,C2,"176.079,53",69.051,"1,38%","9.629,47","9.629,47",True
2,D1,D2,"175.621,13",69.105,"1,38%","9.617,19","9.617,19",True
3,A1,A2,"175.960,34",69.245,"1,38%","9.616,85","9.616,85",True
4,B1,B2,"175.365,86",69.344,"1,39%","9.599,49","9.599,49",True
...,...,...,...,...,...,...,...,...
672,T1,T2,"-75,90",6.620,"0,13%","5.984,08","5.984,08",True
673,M1,E2,"-78,90",6.771,"0,14%","5.983,82","5.983,82",True
674,M1,I2,"-88,75",6.600,"0,13%","5.981,27","5.981,27",True
675,Z1,Z2,"-351.312,04",131.746,"2,63%","2.205,40","2.205,40",True


In [9]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_1 + classe_3
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_classe_1_classe_3 = base_historica.groupby(['classe_1', 'classe_3']).agg({'z_score': 'sum', 'classe_1': 'count', 'valor': 'mean'})
analise_classe_1_classe_3.rename(columns={'z_score': 'soma_z_score', 'classe_1': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_1_classe_3.reset_index(inplace=True)
analise_classe_1_classe_3['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1_classe_3.soma_z_score / analise_classe_1_classe_3.quantidade)
analise_classe_1_classe_3.insert(3, 'proporcao', analise_classe_1_classe_3.quantidade / quantidade_linhas_base_historica)
analise_classe_1_classe_3.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_classe_1_classe_3.proporcao * analise_classe_1_classe_3.media_real).sum()
resultado = 'ela é igual à média da base histórica' if media_ponderada.round(2) == media_base_historica.round(2) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_classe_1_classe_3.soma_z_score.sum()
soma_quantidade = analise_classe_1_classe_3.quantidade.sum()
soma_proporcao = analise_classe_1_classe_3.proporcao.sum()
analise_classe_1_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_3=[''],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_1_classe_3],
    ignore_index=True)

analise_classe_1_classe_3['medias_iguais'] = analise_classe_1_classe_3.media_real.round(0) == analise_classe_1_classe_3.media_calculada_com_z_score.round(0)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_classe_1_classe_3.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_classe_1_classe_3.proporcao = analise_classe_1_classe_3.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_classe_1_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 6.000,40 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_1,classe_3,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,,"-0,00",5.000.000,"100,00%","6.000,40","6.000,40",True
1,C1,C3,"176.072,11",69.184,"1,38%","9.622,34","9.622,34",True
2,A1,A3,"175.930,33",69.246,"1,38%","9.616,18","9.616,18",True
3,D1,D3,"175.593,71",69.195,"1,38%","9.611,92","9.611,92",True
4,B1,B3,"175.427,67",69.137,"1,38%","9.611,54","9.611,54",True
...,...,...,...,...,...,...,...,...
672,H1,N3,"-72,22",6.642,"0,13%","5.984,93","5.984,93",True
673,U1,C3,"-82,66",6.564,"0,13%","5.982,48","5.982,48",True
674,T1,P3,"-94,55",6.707,"0,13%","5.980,34","5.980,34",True
675,Y1,Y3,"-351.359,51",131.846,"2,64%","2.207,77","2.207,77",True


In [10]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_2 + classe_3
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_classe_2_classe_3 = base_historica.groupby(['classe_2', 'classe_3']).agg({'z_score': 'sum', 'classe_2': 'count', 'valor': 'mean'})
analise_classe_2_classe_3.rename(columns={'z_score': 'soma_z_score', 'classe_2': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_2_classe_3.reset_index(inplace=True)
analise_classe_2_classe_3['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_2_classe_3.soma_z_score / analise_classe_2_classe_3.quantidade)
analise_classe_2_classe_3.insert(3, 'proporcao', analise_classe_2_classe_3.quantidade / quantidade_linhas_base_historica)
analise_classe_2_classe_3.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_classe_2_classe_3.proporcao * analise_classe_2_classe_3.media_real).sum()
resultado = 'ela é igual à média da base histórica' if media_ponderada.round(2) == media_base_historica.round(2) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_classe_2_classe_3.soma_z_score.sum()
soma_quantidade = analise_classe_2_classe_3.quantidade.sum()
soma_proporcao = analise_classe_2_classe_3.proporcao.sum()
analise_classe_2_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_2=['Total'],
                      classe_3=[''],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_2_classe_3],
    ignore_index=True)

analise_classe_2_classe_3['medias_iguais'] = analise_classe_2_classe_3.media_real.round(0) == analise_classe_2_classe_3.media_calculada_com_z_score.round(0)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_classe_2_classe_3.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_classe_2_classe_3.proporcao = analise_classe_2_classe_3.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_classe_2_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 6.000,40 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_2,classe_3,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,,"-0,00",5.000.000,"100,00%","6.000,40","6.000,40",True
1,C2,C3,"176.076,69",69.121,"1,38%","9.625,73","9.625,73",True
2,A2,A3,"175.920,40",69.282,"1,39%","9.614,10","9.614,10",True
3,B2,B3,"175.393,46",69.206,"1,38%","9.607,23","9.607,23",True
4,D2,D3,"175.592,15",69.312,"1,39%","9.605,80","9.605,80",True
...,...,...,...,...,...,...,...,...
672,G2,T3,"-70,05",6.665,"0,13%","5.985,44","5.985,44",True
673,A2,V3,"-75,58",6.679,"0,13%","5.984,30","5.984,30",True
674,V2,S3,"-120,68",6.519,"0,13%","5.974,05","5.974,05",True
675,Y2,Y3,"-351.410,54",131.625,"2,63%","2.200,85","2.200,85",True


In [11]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento de todas as classes
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_geral = base_historica.groupby(['classe_1', 'classe_2', 'classe_3']).agg({'z_score': 'sum', 'classe_1': 'count', 'valor': 'mean'})
analise_geral.rename(columns={'z_score': 'soma_z_score', 'classe_1': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_geral.reset_index(inplace=True)
analise_geral['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_geral.soma_z_score / analise_geral.quantidade)
analise_geral.insert(3, 'proporcao', analise_geral.quantidade / quantidade_linhas_base_historica)
analise_geral.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_geral.proporcao * analise_geral.media_real).sum()
resultado = 'ela é igual à média da base histórica' if media_ponderada.round(2) == media_base_historica.round(2) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_geral.soma_z_score.sum()
soma_quantidade = analise_geral.quantidade.sum()
soma_proporcao = analise_geral.proporcao.sum()
analise_geral = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_2=[''],
                      classe_3=[''],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_geral],
    ignore_index=True)

analise_geral['medias_iguais'] = analise_geral.media_real.round(0) == analise_geral.media_calculada_com_z_score.round(0)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_geral.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_geral.proporcao = analise_geral.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_geral.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 6.000,40 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_1,classe_2,classe_3,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,,,"-0,00",5.000.000,"100,00%","6.000,40","6.000,40",True
1,C1,C2,C3,"176.076,49",62.743,"1,25%","9.994,25","9.994,25",True
2,A1,A2,A3,"175.917,21",62.786,"1,26%","9.987,91","9.987,91",True
3,D1,D2,D3,"175.585,85",62.762,"1,26%","9.981,92","9.981,92",True
4,B1,B2,B3,"175.401,41",62.734,"1,25%","9.979,51","9.979,51",True
...,...,...,...,...,...,...,...,...,...
17572,S1,O2,K3,"-18,67",240,"0,00%","5.889,67","5.889,67",True
17573,Y1,U2,E3,"-22,27",277,"0,01%","5.885,99","5.885,99",True
17574,F1,N2,L3,"-22,88",258,"0,01%","5.874,21","5.874,21",True
17575,Y1,Y2,Y3,"-351.377,10",125.272,"2,51%","2.008,54","2.008,54",True


<a id='capitulo2'></a>
# Reajustando a meta com uma nova base

In [12]:
# Cria uma nova base para reajustar a meta
base = base_aleatoria(tamanho=1_000_000,
                      media_normal=7_000)

# Define um valor para a meta e calcula a variação da meta
meta = 7_000
variacao_meta = meta * coeficiente_variacao_base_historica

# Imprime a base na tela
display(base)

Criando os primeiros 900.000 valores
Valores criados em 1,85 segundos

Criando os 25.000 outliers com valores baixos do 1º conjunto de classes
Valores criados em 0,02 segundos

Criando os 25.000 outliers com valores baixos do 2º conjunto de classes
Valores criados em 0,02 segundos

Criando os 12.500 outliers com valores altos do 1º conjunto de classes
Valores criados em 0,02 segundos

Criando os 12.500 outliers com valores altos do 2º conjunto de classes
Valores criados em 0,02 segundos

Criando os 12.500 outliers com valores altos do 3º conjunto de classes
Valores criados em 0,02 segundos

Criando os 12.500 outliers com valores altos do 4º conjunto de classes
Valores criados em 0,02 segundos

Base criada em 1,97 segundos


,classe_1,classe_2,classe_3,valor
0,A1,J2,Q3,6566.38
1,S1,N2,K3,6403.75
2,H1,E2,P3,7208.58
3,U1,U2,M3,5801.65
4,T1,K2,N3,7703.89
...,...,...,...,...
999995,D1,D2,D3,8538.95
999996,D1,D2,D3,11223.36
999997,D1,D2,D3,14257.96
999998,D1,D2,D3,10432.51


In [13]:
# Calcula a média geral do Valor
media_base = base.valor.mean()
print(f'Média do Valor: {media_base:_.2f}'.replace('.', ',').replace('_', '.'))

Média do Valor: 6.901,12


In [14]:
# Resume a base e a base histórica para juntá-las, visando facilitar os cálculos
base = base.groupby(['classe_1', 'classe_2', 'classe_3'], as_index=False).valor.agg(valor='sum', quantidade='count')
base['proporcao'] = base.quantidade / base.quantidade.sum()
base_historica_resumida = base_historica.groupby(['classe_1', 'classe_2', 'classe_3'], as_index=False).z_score.agg(soma_z_score='sum', quantidade_historica='count')
base_historica_resumida['proporcao_historica'] = base_historica_resumida.quantidade_historica / base_historica_resumida.quantidade_historica.sum()
base = base.merge(base_historica_resumida, how='outer', on=['classe_1', 'classe_2', 'classe_3']).fillna(0)
base = base.astype({'valor': float, 'quantidade': int, 'soma_z_score': float, 'quantidade_historica': int, 'proporcao_historica': float})
display(base)

,classe_1,classe_2,classe_3,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica
0,A1,A2,A3,1.254976e+08,12551,0.012551,175917.208381,62786,0.012557
1,A1,A2,B3,2.636082e+05,38,0.000038,-5.317526,259,0.000052
2,A1,A2,C3,3.571780e+05,50,0.000050,5.306943,240,0.000048
3,A1,A2,D3,3.723886e+05,53,0.000053,3.414760,249,0.000050
4,A1,A2,E3,3.630489e+05,52,0.000052,-4.827323,258,0.000052
...,...,...,...,...,...,...,...,...,...
17571,Z1,Z2,V3,2.819598e+05,40,0.000040,-2.398483,254,0.000051
17572,Z1,Z2,W3,3.845472e+05,54,0.000054,1.782804,266,0.000053
17573,Z1,Z2,X3,3.537107e+05,50,0.000050,2.987355,261,0.000052
17574,Z1,Z2,Y3,3.163679e+05,46,0.000046,8.868550,277,0.000055


In [15]:
# Calcula os desvios da classe_1 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_1 = base.groupby('classe_1', as_index=False).sum(numeric_only=True)
analise_classe_1['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1.soma_z_score / analise_classe_1.quantidade_historica)
analise_classe_1['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_1.soma_z_score / analise_classe_1.quantidade_historica)
analise_classe_1['valor_medio'] = base.valor / base.quantidade
analise_classe_1['desvio_valor'] = (analise_classe_1.valor_medio / analise_classe_1.meta_ajustada_com_z_score) - 1
analise_classe_1['desvio_proporcao'] = (analise_classe_1.proporcao / analise_classe_1.proporcao_historica) - 1
analise_classe_1.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_classe_1.proporcao_historica * analise_classe_1.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if meta_ponderada.round(2) == round(float(meta), 2) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_classe_1.valor.sum()
soma_quantidade = analise_classe_1.quantidade.sum()
soma_proporcao = analise_classe_1.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_1.soma_z_score.sum()
soma_quantidade_historica = analise_classe_1.quantidade_historica.sum()
soma_proporcao_historica = analise_classe_1.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_classe_1 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_classe_1],
    ignore_index=True)

analise_classe_1.proporcao = analise_classe_1.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1.proporcao_historica = analise_classe_1.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1.desvio_valor = analise_classe_1.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1.desvio_proporcao = analise_classe_1.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_1.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_1,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,"6.901.118.914,23",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","6.000,40","7.000,00","6.901,12","-1,41%","0,00%"
1,Z1,"292.174.439,97",59.600,"5,96%","-351.219,23",298.357,"5,97%","4.325,08","5.045,59","7.037,56","39,48%","-0,12%"
2,Y1,"292.023.164,25",59.549,"5,95%","-351.348,40",299.026,"5,98%","4.328,21","5.049,24","6.933,82","37,32%","-0,43%"
3,A1,"366.740.893,42",47.013,"4,70%","176.108,41",234.985,"4,70%","7.066,99","8.244,27","9.999,01","21,28%","0,03%"
4,S1,"242.793.481,32",34.693,"3,47%","13,55",172.657,"3,45%","6.000,51","7.000,13","7.121,97","1,74%","0,47%"
5,N1,"242.766.905,51",34.663,"3,47%","82,59",172.945,"3,46%","6.001,08","7.000,79","7.118,23","1,68%","0,21%"
6,L1,"243.450.332,94",34.764,"3,48%","-157,83",174.024,"3,48%","5.999,11","6.998,49","7.106,24","1,54%","-0,12%"
7,F1,"241.695.498,94",34.541,"3,45%","-271,36",173.519,"3,47%","5.998,18","6.997,40","7.100,33","1,47%","-0,47%"
8,U1,"245.483.593,80",35.052,"3,51%","79,55",172.067,"3,44%","6.001,06","7.000,77","7.100,86","1,43%","1,86%"
9,W1,"241.402.848,91",34.513,"3,45%","-35,69",173.841,"3,48%","6.000,11","6.999,66","7.083,94","1,20%","-0,73%"


In [16]:
# Calcula os desvios da classe_2 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_2 = base.groupby('classe_2', as_index=False).sum(numeric_only=True)
analise_classe_2['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_2.soma_z_score / analise_classe_2.quantidade_historica)
analise_classe_2['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_2.soma_z_score / analise_classe_2.quantidade_historica)
analise_classe_2['valor_medio'] = base.valor / base.quantidade
analise_classe_2['desvio_valor'] = (analise_classe_2.valor_medio / analise_classe_2.meta_ajustada_com_z_score) - 1
analise_classe_2['desvio_proporcao'] = (analise_classe_2.proporcao / analise_classe_2.proporcao_historica) - 1
analise_classe_2.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_classe_2.proporcao_historica * analise_classe_2.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if meta_ponderada.round(2) == round(float(meta), 2) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_classe_2.valor.sum()
soma_quantidade = analise_classe_2.quantidade.sum()
soma_proporcao = analise_classe_2.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_2.soma_z_score.sum()
soma_quantidade_historica = analise_classe_2.quantidade_historica.sum()
soma_proporcao_historica = analise_classe_2.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_classe_2 = pd.concat([
    pd.DataFrame(dict(classe_2=['Total'],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_classe_2],
    ignore_index=True)

analise_classe_2.proporcao = analise_classe_2.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_2.proporcao_historica = analise_classe_2.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_2.desvio_valor = analise_classe_2.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_2.desvio_proporcao = analise_classe_2.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_2.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_2,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,"6.901.118.914,23",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","6.000,40","7.000,00","6.901,12","-1,41%","-0,00%"
1,Z2,"291.166.237,66",59.441,"5,94%","-351.106,84",298.331,"5,97%","4.325,47","5.046,04","7.037,56","39,47%","-0,38%"
2,Y2,"290.961.653,31",59.421,"5,94%","-351.535,16",298.145,"5,96%","4.322,38","5.042,44","6.933,82","37,51%","-0,35%"
3,A2,"366.544.479,42",46.975,"4,70%","175.763,79",236.079,"4,72%","7.059,97","8.236,08","9.999,01","21,40%","-0,51%"
4,S2,"242.854.751,09",34.696,"3,47%","22,06",172.437,"3,45%","6.000,58","7.000,21","7.121,97","1,74%","0,60%"
5,N2,"244.149.504,52",34.891,"3,49%","-11,70",173.574,"3,47%","6.000,31","6.999,89","7.118,23","1,69%","0,51%"
6,L2,"241.424.206,79",34.460,"3,45%","-18,05",172.552,"3,45%","6.000,25","6.999,83","7.106,24","1,52%","-0,15%"
7,U2,"241.131.254,26",34.439,"3,44%","9,57",173.168,"3,46%","6.000,48","7.000,09","7.100,86","1,44%","-0,56%"
8,F2,"242.739.803,84",34.677,"3,47%","4,74",173.420,"3,47%","6.000,44","7.000,05","7.100,33","1,43%","-0,02%"
9,W2,"243.587.055,74",34.795,"3,48%","57,38",173.408,"3,47%","6.000,87","7.000,55","7.083,94","1,19%","0,33%"


In [17]:
# Calcula os desvios da classe_3 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_3 = base.groupby('classe_3', as_index=False).sum(numeric_only=True)
analise_classe_3['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_3.soma_z_score / analise_classe_3.quantidade_historica)
analise_classe_3['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_3.soma_z_score / analise_classe_3.quantidade_historica)
analise_classe_3['valor_medio'] = base.valor / base.quantidade
analise_classe_3['desvio_valor'] = (analise_classe_3.valor_medio / analise_classe_3.meta_ajustada_com_z_score) - 1
analise_classe_3['desvio_proporcao'] = (analise_classe_3.proporcao / analise_classe_3.proporcao_historica) - 1
analise_classe_3.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_classe_3.proporcao_historica * analise_classe_3.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if meta_ponderada.round(2) == round(float(meta), 2) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_classe_3.valor.sum()
soma_quantidade = analise_classe_3.quantidade.sum()
soma_proporcao = analise_classe_3.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_3.soma_z_score.sum()
soma_quantidade_historica = analise_classe_3.quantidade_historica.sum()
soma_proporcao_historica = analise_classe_3.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_3=['Total'],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_classe_3],
    ignore_index=True)

analise_classe_3.proporcao = analise_classe_3.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_3.proporcao_historica = analise_classe_3.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_3.desvio_valor = analise_classe_3.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_3.desvio_proporcao = analise_classe_3.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_3,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,"6.901.118.914,23",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","6.000,40","7.000,00","6.901,12","-1,41%","0,00%"
1,Z3,"292.465.818,54",59.622,"5,96%","-351.336,85",298.083,"5,96%","4.322,98","5.043,14","7.037,56","39,55%","0,01%"
2,Y3,"293.031.568,74",59.710,"5,97%","-351.247,76",298.217,"5,96%","4.324,16","5.044,51","6.933,82","37,45%","0,11%"
3,A3,"369.063.385,01",47.352,"4,74%","175.797,85",235.853,"4,72%","7.061,19","8.237,50","9.999,01","21,38%","0,38%"
4,S3,"239.333.511,04",34.214,"3,42%","69,29",172.599,"3,45%","6.000,97","7.000,67","7.121,97","1,73%","-0,89%"
5,N3,"243.247.653,87",34.731,"3,47%","-189,09",173.057,"3,46%","5.998,85","6.998,19","7.118,23","1,72%","0,35%"
6,L3,"240.656.310,53",34.378,"3,44%","125,50",172.513,"3,45%","6.001,44","7.001,21","7.106,24","1,50%","-0,36%"
7,U3,"242.481.954,75",34.608,"3,46%","-65,65",172.950,"3,46%","5.999,86","6.999,37","7.100,86","1,45%","0,05%"
8,F3,"242.525.924,02",34.637,"3,46%","-39,73",173.240,"3,46%","6.000,08","6.999,62","7.100,33","1,44%","-0,03%"
9,W3,"243.811.236,50",34.805,"3,48%","-155,38",173.367,"3,47%","5.999,13","6.998,51","7.083,94","1,22%","0,38%"


In [18]:
# Calcula os desvios da classe_1 + classe_2 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_1_classe_2 = base.groupby(['classe_1', 'classe_2'], as_index=False).sum(numeric_only=True)
analise_classe_1_classe_2['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1_classe_2.soma_z_score / analise_classe_1_classe_2.quantidade_historica)
analise_classe_1_classe_2['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_1_classe_2.soma_z_score / analise_classe_1_classe_2.quantidade_historica)
analise_classe_1_classe_2['valor_medio'] = base.valor / base.quantidade
analise_classe_1_classe_2['desvio_valor'] = (analise_classe_1_classe_2.valor_medio / analise_classe_1_classe_2.meta_ajustada_com_z_score) - 1
analise_classe_1_classe_2['desvio_proporcao'] = (analise_classe_1_classe_2.proporcao / analise_classe_1_classe_2.proporcao_historica) - 1
analise_classe_1_classe_2.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_classe_1_classe_2.proporcao_historica * analise_classe_1_classe_2.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if meta_ponderada.round(2) == round(float(meta), 2) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_classe_1_classe_2.valor.sum()
soma_quantidade = analise_classe_1_classe_2.quantidade.sum()
soma_proporcao = analise_classe_1_classe_2.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_1_classe_2.soma_z_score.sum()
soma_quantidade_historica = analise_classe_1_classe_2.quantidade_historica.sum()
soma_proporcao_historica = analise_classe_1_classe_2.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_classe_1_classe_2 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_2=[''],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_classe_1_classe_2],
    ignore_index=True)

analise_classe_1_classe_2.proporcao = analise_classe_1_classe_2.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1_classe_2.proporcao_historica = analise_classe_1_classe_2.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1_classe_2.desvio_valor = analise_classe_1_classe_2.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1_classe_2.desvio_proporcao = analise_classe_1_classe_2.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_1_classe_2.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_1,classe_2,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,,"6.901.118.914,23",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","6.000,40","7.000,00","6.901,12","-1,41%","0,00%"
1,Z1,Z2,"59.666.997,55",26.382,"2,64%","-351.312,04",131.746,"2,63%","2.205,40","2.572,80","7.142,57","177,62%","0,12%"
2,Y1,Y2,"59.339.527,56",26.330,"2,63%","-351.382,37",131.725,"2,63%","2.204,04","2.571,21","6.971,71","171,15%","-0,06%"
3,M1,A2,"9.133.780,42",1.308,"0,13%","-13,28",6.715,"0,13%","5.997,59","6.996,72","7.213,67","3,10%","-2,61%"
4,F1,H2,"9.233.918,65",1.324,"0,13%","-42,07",6.694,"0,13%","5.991,46","6.989,57","7.204,06","3,07%","-1,11%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,V1,L2,"8.853.920,62",1.267,"0,13%","-27,84",6.687,"0,13%","5.994,48","6.993,09","6.768,96","-3,21%","-5,26%"
673,A1,A2,"133.977.311,89",13.758,"1,38%","175.960,34",69.245,"1,38%","9.616,85","11.218,91","9.999,01","-10,87%","-0,66%"
674,C1,C2,"134.275.580,32",13.816,"1,38%","176.079,53",69.051,"1,38%","9.629,47","11.233,63","7.022,89","-37,48%","0,04%"
675,B1,B2,"134.268.837,99",13.778,"1,38%","175.365,86",69.344,"1,39%","9.599,49","11.198,65","6.956,80","-37,88%","-0,65%"


In [19]:
# Calcula os desvios da classe_1 + classe_3 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_1_classe_3 = base.groupby(['classe_1', 'classe_3'], as_index=False).sum(numeric_only=True)
analise_classe_1_classe_3['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1_classe_3.soma_z_score / analise_classe_1_classe_3.quantidade_historica)
analise_classe_1_classe_3['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_1_classe_3.soma_z_score / analise_classe_1_classe_3.quantidade_historica)
analise_classe_1_classe_3['valor_medio'] = base.valor / base.quantidade
analise_classe_1_classe_3['desvio_valor'] = (analise_classe_1_classe_3.valor_medio / analise_classe_1_classe_3.meta_ajustada_com_z_score) - 1
analise_classe_1_classe_3['desvio_proporcao'] = (analise_classe_1_classe_3.proporcao / analise_classe_1_classe_3.proporcao_historica) - 1
analise_classe_1_classe_3.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_classe_1_classe_3.proporcao_historica * analise_classe_1_classe_3.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if meta_ponderada.round(2) == round(float(meta), 2) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_classe_1_classe_3.valor.sum()
soma_quantidade = analise_classe_1_classe_3.quantidade.sum()
soma_proporcao = analise_classe_1_classe_3.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_1_classe_3.soma_z_score.sum()
soma_quantidade_historica = analise_classe_1_classe_3.quantidade_historica.sum()
soma_proporcao_historica = analise_classe_1_classe_3.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_classe_1_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_3=[''],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_classe_1_classe_3],
    ignore_index=True)

analise_classe_1_classe_3.proporcao = analise_classe_1_classe_3.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1_classe_3.proporcao_historica = analise_classe_1_classe_3.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1_classe_3.desvio_valor = analise_classe_1_classe_3.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1_classe_3.desvio_proporcao = analise_classe_1_classe_3.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_1_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_1,classe_3,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,,"6.901.118.914,23",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","6.000,40","7.000,00","6.901,12","-1,41%","0,00%"
1,Z1,Z3,"59.225.577,84",26.317,"2,63%","-351.324,04",131.560,"2,63%","2.199,91","2.566,39","7.142,57","178,31%","0,02%"
2,Y1,Y3,"58.843.694,25",26.254,"2,63%","-351.359,51",131.846,"2,64%","2.207,77","2.575,56","6.971,71","170,69%","-0,44%"
3,F1,H3,"9.812.380,48",1.403,"0,14%","-26,99",6.739,"0,13%","5.994,70","6.993,35","7.204,06","3,01%","4,10%"
4,L1,H3,"9.458.132,09",1.352,"0,14%","-26,55",6.717,"0,13%","5.994,78","6.993,44","7.202,68","2,99%","0,64%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,V1,L3,"9.011.933,17",1.286,"0,13%","-34,54",6.633,"0,13%","5.992,99","6.991,35","6.768,96","-3,18%","-3,06%"
673,A1,A3,"134.378.969,64",13.818,"1,38%","175.930,33",69.246,"1,38%","9.616,18","11.218,13","9.999,01","-10,87%","-0,23%"
674,C1,C3,"134.402.533,67",13.831,"1,38%","176.072,11",69.184,"1,38%","9.622,34","11.225,31","7.022,89","-37,44%","-0,04%"
675,B1,B3,"134.496.429,33",13.817,"1,38%","175.427,67",69.137,"1,38%","9.611,54","11.212,71","6.956,80","-37,96%","-0,08%"


In [20]:
# Calcula os desvios da classe_2 + classe_3 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_2_classe_3 = base.groupby(['classe_2', 'classe_3'], as_index=False).sum(numeric_only=True)
analise_classe_2_classe_3['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_2_classe_3.soma_z_score / analise_classe_2_classe_3.quantidade_historica)
analise_classe_2_classe_3['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_2_classe_3.soma_z_score / analise_classe_2_classe_3.quantidade_historica)
analise_classe_2_classe_3['valor_medio'] = base.valor / base.quantidade
analise_classe_2_classe_3['desvio_valor'] = (analise_classe_2_classe_3.valor_medio / analise_classe_2_classe_3.meta_ajustada_com_z_score) - 1
analise_classe_2_classe_3['desvio_proporcao'] = (analise_classe_2_classe_3.proporcao / analise_classe_2_classe_3.proporcao_historica) - 1
analise_classe_2_classe_3.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_classe_2_classe_3.proporcao_historica * analise_classe_2_classe_3.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if meta_ponderada.round(2) == round(float(meta), 2) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_classe_2_classe_3.valor.sum()
soma_quantidade = analise_classe_2_classe_3.quantidade.sum()
soma_proporcao = analise_classe_2_classe_3.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_2_classe_3.soma_z_score.sum()
soma_quantidade_historica = analise_classe_2_classe_3.quantidade_historica.sum()
soma_proporcao_historica = analise_classe_2_classe_3.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_classe_2_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_2=['Total'],
                      classe_3=[''],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_classe_2_classe_3],
    ignore_index=True)

analise_classe_2_classe_3.proporcao = analise_classe_2_classe_3.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_2_classe_3.proporcao_historica = analise_classe_2_classe_3.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_2_classe_3.desvio_valor = analise_classe_2_classe_3.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_2_classe_3.desvio_proporcao = analise_classe_2_classe_3.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_2_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_2,classe_3,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,,"6.901.118.914,23",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","6.000,40","7.000,00","6.901,12","-1,41%","0,00%"
1,Z2,Z3,"59.434.177,62",26.349,"2,63%","-351.415,16",131.593,"2,63%","2.199,88","2.566,35","7.142,57","178,32%","0,12%"
2,Y2,Y3,"59.395.259,36",26.338,"2,63%","-351.410,54",131.625,"2,63%","2.200,85","2.567,49","6.971,71","171,54%","0,05%"
3,G2,O3,"9.127.367,01",1.299,"0,13%","-39,10",6.729,"0,13%","5.992,13","6.990,35","7.199,26","2,99%","-3,48%"
4,L2,H3,"9.361.432,00",1.335,"0,13%","-18,22",6.746,"0,13%","5.996,56","6.995,52","7.202,68","2,96%","-1,05%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,V2,L3,"9.413.560,41",1.348,"0,13%","33,38",6.603,"0,13%","6.007,60","7.008,39","6.768,96","-3,42%","2,07%"
673,A2,A3,"134.715.925,89",13.870,"1,39%","175.920,40",69.282,"1,39%","9.614,10","11.215,70","9.999,01","-10,85%","0,10%"
674,C2,C3,"134.608.644,26",13.855,"1,39%","176.076,69",69.121,"1,38%","9.625,73","11.229,27","7.022,89","-37,46%","0,22%"
675,B2,B3,"134.807.793,57",13.862,"1,39%","175.393,46",69.206,"1,38%","9.607,23","11.207,69","6.956,80","-37,93%","0,15%"


In [21]:
# Calcula os desvios de todas as classes em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_geral = base.groupby(['classe_1', 'classe_2', 'classe_3'], as_index=False).sum(numeric_only=True)
analise_geral['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_geral.soma_z_score / analise_geral.quantidade_historica)
analise_geral['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_geral.soma_z_score / analise_geral.quantidade_historica)
analise_geral['valor_medio'] = base.valor / base.quantidade
analise_geral['desvio_valor'] = (analise_geral.valor_medio / analise_geral.meta_ajustada_com_z_score) - 1
analise_geral['desvio_proporcao'] = (analise_geral.proporcao / analise_geral.proporcao_historica) - 1
analise_geral.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_geral.proporcao_historica * analise_geral.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if meta_ponderada.round(2) == round(float(meta), 2) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_geral.valor.sum()
soma_quantidade = analise_geral.quantidade.sum()
soma_proporcao = analise_geral.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_geral.soma_z_score.sum()
soma_quantidade_historica = analise_geral.quantidade_historica.sum()
soma_proporcao_historica = analise_geral.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_geral = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_2=[''],
                      classe_3=[''],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_geral],
    ignore_index=True)

analise_geral.proporcao = analise_geral.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_geral.proporcao_historica = analise_geral.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_geral.desvio_valor = analise_geral.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_geral.desvio_proporcao = analise_geral.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_geral.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_1,classe_2,classe_3,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,,,"6.901.118.914,23",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","6.000,40","7.000,00","6.901,12","-1,41%","-0,00%"
1,I1,L2,F3,"256.354,12",35,"0,00%","-0,87",257,"0,01%","5.995,56","6.994,35","7.324,40","4,72%","-31,91%"
2,G1,L2,U3,"418.742,98",58,"0,01%","-11,95",264,"0,01%","5.935,99","6.924,86","7.219,71","4,26%","9,85%"
3,D1,Y2,T3,"345.949,62",48,"0,00%","-12,69",256,"0,01%","5.929,87","6.917,72","7.207,28","4,19%","-6,25%"
4,Z1,R2,N3,"310.096,84",43,"0,00%","-11,01",242,"0,00%","5.935,67","6.924,49","7.211,55","4,15%","-11,16%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17572,Z1,Z2,Z3,"50.419.993,27",25.062,"2,51%","-351.372,51",125.265,"2,51%","2.008,37","2.342,94","2.011,81","-14,13%","0,04%"
17573,A1,A2,A3,"125.497.555,59",12.551,"1,26%","175.917,21",62.786,"1,26%","9.987,91","11.651,78","9.999,01","-14,18%","-0,05%"
17574,Y1,Y2,Y3,"50.332.488,94",25.044,"2,50%","-351.377,10",125.272,"2,51%","2.008,54","2.343,14","2.009,76","-14,23%","-0,04%"
17575,C1,C2,C3,"125.541.006,78",12.561,"1,26%","176.076,49",62.743,"1,25%","9.994,25","11.659,18","9.994,51","-14,28%","0,10%"


# Fim